In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Set to True to force a full refresh of the data
full_refresh = True

In [3]:
repo_root = os.popen('git rev-parse --show-toplevel').read().strip()

In [4]:
data_folder = os.path.join(repo_root, 'multi-parameter-estimation', 'data')

# Get list of data directories
data_dirs = os.listdir(data_folder)
data_dirs = [d for d in data_dirs if os.path.isdir(os.path.join(data_folder, d))]

# skip old-data
if 'old-data' in data_dirs:
    data_dirs.remove('old-data')

data_dirs.sort()
data_dirs

['2025-05-23--17h-24m-53s',
 '2025-05-23--17h-27m-00s',
 '2025-05-23--17h-28m-51s',
 '2025-05-23--17h-30m-41s',
 '2025-05-23--17h-32m-32s',
 '2025-05-23--17h-34m-23s',
 '2025-05-23--17h-36m-14s',
 '2025-05-23--17h-38m-05s',
 '2025-05-23--17h-39m-56s',
 '2025-05-23--17h-41m-47s',
 '2025-05-23--17h-43m-38s',
 '2025-05-23--17h-45m-29s',
 '2025-05-23--17h-47m-20s',
 '2025-05-23--17h-49m-11s',
 '2025-05-23--17h-51m-02s',
 '2025-05-23--17h-52m-53s',
 '2025-05-23--17h-54m-44s',
 '2025-05-23--17h-56m-35s',
 '2025-05-23--17h-58m-27s',
 '2025-05-23--18h-00m-17s',
 '2025-05-23--18h-02m-09s',
 '2025-05-23--18h-04m-00s',
 '2025-05-23--18h-05m-51s',
 '2025-05-23--18h-07m-42s',
 '2025-05-23--18h-09m-34s',
 '2025-05-23--18h-11m-25s',
 '2025-05-23--18h-13m-17s',
 '2025-05-23--18h-15m-08s',
 '2025-05-23--18h-16m-59s',
 '2025-05-23--18h-18m-47s',
 '2025-05-23--18h-20m-26s',
 '2025-05-23--18h-22m-05s',
 '2025-05-23--18h-23m-43s',
 '2025-05-23--18h-25m-22s',
 '2025-05-23--18h-27m-01s',
 '2025-05-23--18h-28

In [5]:
new_data_dirs = data_dirs.copy()

if not full_refresh:
    for d in data_dirs:
        if os.path.exists(os.path.join(data_folder, d, "chunked_coincidences_n=200.csv")):
            new_data_dirs.remove(d)

new_data_dirs

['2025-05-23--17h-24m-53s',
 '2025-05-23--17h-27m-00s',
 '2025-05-23--17h-28m-51s',
 '2025-05-23--17h-30m-41s',
 '2025-05-23--17h-32m-32s',
 '2025-05-23--17h-34m-23s',
 '2025-05-23--17h-36m-14s',
 '2025-05-23--17h-38m-05s',
 '2025-05-23--17h-39m-56s',
 '2025-05-23--17h-41m-47s',
 '2025-05-23--17h-43m-38s',
 '2025-05-23--17h-45m-29s',
 '2025-05-23--17h-47m-20s',
 '2025-05-23--17h-49m-11s',
 '2025-05-23--17h-51m-02s',
 '2025-05-23--17h-52m-53s',
 '2025-05-23--17h-54m-44s',
 '2025-05-23--17h-56m-35s',
 '2025-05-23--17h-58m-27s',
 '2025-05-23--18h-00m-17s',
 '2025-05-23--18h-02m-09s',
 '2025-05-23--18h-04m-00s',
 '2025-05-23--18h-05m-51s',
 '2025-05-23--18h-07m-42s',
 '2025-05-23--18h-09m-34s',
 '2025-05-23--18h-11m-25s',
 '2025-05-23--18h-13m-17s',
 '2025-05-23--18h-15m-08s',
 '2025-05-23--18h-16m-59s',
 '2025-05-23--18h-18m-47s',
 '2025-05-23--18h-20m-26s',
 '2025-05-23--18h-22m-05s',
 '2025-05-23--18h-23m-43s',
 '2025-05-23--18h-25m-22s',
 '2025-05-23--18h-27m-01s',
 '2025-05-23--18h-28

In [6]:
def load_chunks(data_dir):
    if not os.path.exists(os.path.join(data_folder, data_dir, "chunked_coincidences_n=40.csv")):
        print(f"Skipping {data_dir} as it does not have the required file.")
        return pd.DataFrame()
    coincidences = pd.read_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=40.csv"))
    coincidences["data_dir"] = data_dir
    return coincidences

chunks_df = pd.concat([load_chunks(d) for d in new_data_dirs], ignore_index=True)
chunks_df 

,data_dir,C,DB_H,DB_V,SB,N
0,2025-05-23--17h-24m-53s,2.00,38.0,0.0,0.00,40.0
1,2025-05-23--17h-24m-53s,0.75,39.0,0.0,0.25,40.0
2,2025-05-23--17h-24m-53s,1.00,39.0,0.0,0.00,40.0
3,2025-05-23--17h-24m-53s,0.75,39.0,0.0,0.25,40.0
4,2025-05-23--17h-24m-53s,1.00,39.0,0.0,0.00,40.0
...,...,...,...,...,...,...
5055,2025-05-23--19h-08m-07s,0.00,0.0,40.0,0.00,40.0
5056,2025-05-23--19h-08m-07s,1.00,0.0,39.0,0.00,40.0
5057,2025-05-23--19h-08m-07s,1.00,0.0,39.0,0.00,40.0
5058,2025-05-23--19h-08m-07s,1.00,0.0,39.0,0.00,40.0


In [7]:
# drop columns that are not needed
def k_wise_sum(group, k):
    # Drop last rows if not divisible by k
    n = len(group) - (len(group) % k)
    group = group.iloc[:n].reset_index(drop=True)
    # Sum every k rows
    kwise = group.groupby(group.index // k).sum()
    # Restore data_dir from the first row of each group
    kwise['data_dir'] = group['data_dir'].iloc[::k].values
    return kwise

# Example usage for k=3
df_80 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=2)).reset_index(drop=True)
df_120 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=3)).reset_index(drop=True)
df_160 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=4)).reset_index(drop=True)
df_200 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=5)).reset_index(drop=True)
df_200


/tmp/ipykernel_1148624/345050552.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_80 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=2)).reset_index(drop=True)
/tmp/ipykernel_1148624/345050552.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_120 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=3)).reset_index(drop=True)
/tmp/ip

,data_dir,C,DB_H,DB_V,SB,N
0,2025-05-23--17h-24m-53s,5.50,194.0,0.0,0.50,200.0
1,2025-05-23--17h-24m-53s,3.00,197.0,0.0,0.00,200.0
2,2025-05-23--17h-24m-53s,2.25,197.0,0.0,0.75,200.0
3,2025-05-23--17h-24m-53s,4.75,195.0,0.0,0.25,200.0
4,2025-05-23--17h-24m-53s,2.50,197.0,0.0,0.50,200.0
...,...,...,...,...,...,...
980,2025-05-23--19h-08m-07s,3.75,0.0,196.0,0.25,200.0
981,2025-05-23--19h-08m-07s,3.50,0.0,196.0,0.50,200.0
982,2025-05-23--19h-08m-07s,2.75,0.0,197.0,0.25,200.0
983,2025-05-23--19h-08m-07s,4.25,0.0,195.0,0.75,200.0


In [8]:
# save the dataframes to csv files
# save the dataframes to csv files based on the data_dir
for data_dir in new_data_dirs:
    df_80_subset = df_80[df_80["data_dir"] == data_dir]
    df_120_subset = df_120[df_120["data_dir"] == data_dir]
    df_160_subset = df_160[df_160["data_dir"] == data_dir]
    df_200_subset = df_200[df_200["data_dir"] == data_dir]
    df_80_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=80.csv"), index=False)
    df_120_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=120.csv"), index=False)
    df_160_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=160.csv"), index=False)
    df_200_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=200.csv"), index=False)